# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd


In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", result_count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame

# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Execute the query
cursor = establishments.find(query)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(cursor))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.


# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint
query = {'LocalAuthorityName': {'$regex' : 'London'} , 'RatingValue': {'$gte' : '4'}}



# Use count_documents to display the number of documents in the result

rating_value = establishments.count_documents(query)
print(f'There are {rating_value} restaurants with a rating value greater than or equal to 4.')



# Display the first document in the results using pprint
pprint(establishments.find_one(query))


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
# Convert the result to a Pandas DataFrame
query = {'LocalAuthorityName': {'$regex' : 'London'} , 'RatingValue': {'$gte' : '4'}}
results = establishments.find(query)
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame

num_rows = df['_id'].count()
print(num_rows)

# Display the first 10 rows of the DataFrame
df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [31]:
fields = {'geocode': 1}

establishments.find_one({'geocode.latitude': {'$gte' : 5}} ,fields)


{'_id': ObjectId('652f2ce923865a58dada0235'),
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812}}

In [32]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# degree_search = 0.01
# latitude = 
# longitude = 

# query = 
# sort =  

# Print the results

degree_search = 0.01
latitude = 51.083812  # Replace with the latitude of Penang Flavours
longitude = 1.195625  # Replace with the longitude of Penang Flavours

# Define the query
query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

# Define the sort criteria
sort = [("scores.Hygiene", 1)]

# Execute the query and sort the results
cursor = establishments.find(query).sort(sort).limit(5)

# Iterate over the cursor and print the results
for doc in cursor:
    pprint(doc)


In [33]:
# Convert result to Pandas DataFrame
cursor = establishments.find(query).sort(sort).limit(5)

# Convert the cursor to a list of dictionaries
results = list(cursor)

# Create a Pandas DataFrame from the list of dictionaries
new_df = pd.DataFrame(results)

# Display the Pandas DataFrame
new_df


""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [34]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
pipeline = [
    {
        "$match": {
            "scores.Hygiene": 0
        }
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Total number of establishments with a hygiene score of 0: {len(results)}")

# Print the first 10 results
for result in results[:10]:
    pprint(result)


Total number of establishments with a hygiene score of 0: 55
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 886}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [35]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
import pandas as pd

# Convert the result to a Pandas DataFrame
final_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
num_rows = len(final_df)
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
first_10_rows = final_df.head(10)
print(first_10_rows)


Number of rows in the DataFrame: 55
               _id  count
0           Thanet   1130
1        Greenwich    886
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
